# Chat with PDF page images

**If you're looking or the web application, check the src/ folder.** 

This notebook demonstrates how to convert PDF pages to images and send them to a vision model for inference

## Authenticate to OpenAI

The following code connects to OpenAI, either using an Azure OpenAI account, GitHub models, or local Ollama model. See the README for instruction on configuring the `.env` file.

In [1]:
import os

import azure.identity
import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_host = os.getenv("OPENAI_HOST", "github")

if openai_host == "github":
    print("Using GitHub Models with GITHUB_TOKEN as key")
    openai_client = openai.OpenAI(
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference",
    )
    model_name = os.getenv("OPENAI_MODEL", "openai/gpt-4o")
elif openai_host == "local":
    print("Using local OpenAI-compatible API with no key")
    openai_client = openai.OpenAI(api_key="no-key-required", base_url=os.environ["LOCAL_OPENAI_ENDPOINT"])
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_KEY_FOR_CHATVISION"):
    # Authenticate using an Azure OpenAI API key
    # This is generally discouraged, but is provided as a convenience
    print("Using Azure OpenAI with key")
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=os.environ["AZURE_OPENAI_KEY_FOR_CHATVISION"],
    )
    # This is actually the deployment name, not the model name
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_ENDPOINT"):
    tenant_id = os.environ["AZURE_TENANT_ID"]
    print("Using Azure OpenAI with Azure Developer CLI credential for tenant id", tenant_id)
    default_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=tenant_id)
    token_provider = azure.identity.get_bearer_token_provider(
        default_credential, "https://cognitiveservices.azure.com/.default"
    )
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=token_provider,
    )
    # This is actually the deployment name, not the model name
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")

print(f"Using model {model_name}")

Using GitHub Models with GITHUB_TOKEN as key
Using model openai/gpt-4o


## Convert PDFs to images

In [2]:
%pip install Pillow PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 36.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.2 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pymupdf
from PIL import Image

filename = "plants.pdf"
doc = pymupdf.open(filename)
for i in range(doc.page_count):
    doc = pymupdf.open(filename)
    page = doc.load_page(i)
    pix = page.get_pixmap()
    original_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    original_img.save(f"page_{i}.png")

## Send images to vision model

In [4]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [6]:
user_content = [{"text": "What plants are listed on these pages?", "type": "text"}]
# Process just the first few pages, as processing all doc.page_count pages is slow
for i in range(3):
    user_content.append({"image_url": {"url": open_image_as_base64(f"page_{i}.png")}, "type": "image_url"})

response = openai_client.chat.completions.create(
    model=model_name, messages=[{"role": "user", "content": user_content}]
)

print(response.choices[0].message.content)

These pages list a variety of plants offered by The Watershed Nursery. Here's a summary of the plant types listed on the individual pages:

---

### **Page 1:**

**Annuals:**
- _Centromadia pungens_ (Common tarweed)
- _Epilobium densiflorum_ (Dense Spike-primrose)
- _Eschscholzia caespitosa_ (Tufted Poppy)
- _Eschscholzia californica_ (California Poppy)
- _Eschscholzia californica 'Purple Gleam'_ (Purple Gleam Poppy)
- _Eschscholzia californica var. maritima_ (Coastal California Poppy)
- _Madia elegans_ (Tarweed)
- _Mentzelia lindleyi_ (Lindley's Blazing Star)
- _Symphyotrichum subulatum_ (Slim marsh aster)
- _Trichostema lanceolatum_ (Vinegar weed)

**Bulbs:**
- _Brodiaea californica_ (California brodiaea)
- _Chlorogalum pomeridianum_ (Soap plant)
- _Epipactis gigantea_ (Stream orchid)
- _Wyethia angustifolia_ (Narrowleaf mule ears)
- _Wyethia mollis_ (Woolly Mule's Ear)

**Grasses:**
- _Agrostis pallens_ (Thingrass)
- _Anthoxanthum occidentale_ (Vanilla grass)
- _Bouteloua gracilis_ 